# Computational Mathematics — Assignment 2
## Solution of a System of Linear Algebraic Equations
**Name:** Noyan Inayatulla


### Selected system (System #2)

\[
\begin{cases}
x_1-6x_2+x_3-x_4=-5\\
-7x_1+x_2-x_3+2x_4=-3\\
x_1+x_2+8x_3-x_4=9\\
-2x_1+x_2-x_3+7x_4=8
\end{cases}
\]


In [3]:
import numpy as np
import pandas as pd

# -------------------------
# System #2
# -------------------------

A_orig = np.array([
    [ 1, -6,  1, -1],
    [-7,  1, -1,  2],
    [ 1,  1,  8, -1],
    [-2,  1, -1,  7]
], dtype=float)

b_orig = np.array([-5, -3, 9, 8], dtype=float)

# -------------------------
# Reorder rows for diagonal dominance (for iterative methods)
# order: (2nd, 1st, 3rd, 4th)
# -------------------------
perm = [1, 0, 2, 3]
A = A_orig[perm, :]
b = b_orig[perm]

# Iterative settings (same for Jacobi/GS/SOR)
tol = 1e-6
max_iter = 100
x0 = np.zeros(4, dtype=float)

def is_diagonally_dominant(A):
    A = np.array(A, dtype=float)
    n = A.shape[0]
    for i in range(n):
        diag = abs(A[i, i])
        off = np.sum(np.abs(A[i, :])) - diag
        if diag <= off:
            return False
    return True

print("A (reordered) =\n", A)
print("b (reordered) =\n", b)
print("Diagonal dominance:", is_diagonally_dominant(A))


A (reordered) =
 [[-7.  1. -1.  2.]
 [ 1. -6.  1. -1.]
 [ 1.  1.  8. -1.]
 [-2.  1. -1.  7.]]
b (reordered) =
 [-3. -5.  9.  8.]
Diagonal dominance: True


In [4]:
def rel_error_inf(x_new, x_old, eps=1e-12):
    num = np.max(np.abs(x_new - x_old))
    den = np.max(np.abs(x_new))
    if den < eps:
        den = 1.0
    return num / den

def residual_inf(A, x, b):
    r = A @ x - b
    return np.max(np.abs(r))

def make_iteration_table(history, errors):
    # history: list of vectors x^(k) including k=0
    # errors: list of errors for k=1..
    rows = []
    for k, xk in enumerate(history):
        err = None
        if k == 0:
            err = np.nan
        else:
            err = errors[k-1]
        rows.append([k, xk[0], xk[1], xk[2], xk[3], err])
    df = pd.DataFrame(rows, columns=["k", "x1", "x2", "x3", "x4", "rel_error"])
    return df

def show_table(df, max_rows=15):
    # prints output in notebook
    if len(df) <= max_rows:
        display(df)
    else:
        display(df.head(max_rows))
        print(f"... table truncated (shown first {max_rows} of {len(df)} rows)")


# Method 1: Cramer's method


In [5]:
def cramer_solve(A, b, eps=1e-12):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    n = A.shape[0]

    detA = np.linalg.det(A)
    if abs(detA) < eps:
        raise ValueError("Cramer's method fails: det(A) is zero (or near zero).")

    x = np.zeros(n, dtype=float)
    for i in range(n):
        Ai = A.copy()
        Ai[:, i] = b
        x[i] = np.linalg.det(Ai) / detA
    return x, detA

x_cramer, detA = cramer_solve(A_orig, b_orig)
print("Cramer solution x =", x_cramer)
print("det(A) =", detA)
print("Residual ||Ax-b||_inf =", residual_inf(A_orig, x_cramer, b_orig))


Cramer solution x = [0.80338983 0.91525424 1.08474576 1.39661017]
det(A) = -2064.999999999999
Residual ||Ax-b||_inf = 7.105427357601002e-15


### Reflection (Cramer’s Method)
Cramer’s method computes each variable using determinants. It works only if det(A) != 0. The main limitation is computational cost: determinant computations become very expensive for larger systems. Therefore, it is mainly used for small systems or theoretical purposes, not for large-scale practical computations.


# Method 2: Gaussian method

In [6]:
def gaussian_solve(A, b, eps=1e-12):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    n = A.shape[0]

    Ab = np.hstack([A, b.reshape(-1, 1)])

    # Forward elimination with partial pivoting
    for col in range(n):
        # find row with max abs value in current column
        pivot_row = col + np.argmax(np.abs(Ab[col:, col]))
        if abs(Ab[pivot_row, col]) < eps:
            raise ValueError("Gaussian method fails: pivot is zero (or near zero).")

        if pivot_row != col:
            Ab[[col, pivot_row]] = Ab[[pivot_row, col]]

        for row in range(col + 1, n):
            factor = Ab[row, col] / Ab[col, col]
            Ab[row, col:] -= factor * Ab[col, col:]

    # back substitution
    x = np.zeros(n, dtype=float)
    for i in range(n - 1, -1, -1):
        rhs = Ab[i, -1] - np.dot(Ab[i, i+1:n], x[i+1:n])
        if abs(Ab[i, i]) < eps:
            raise ValueError("Gaussian method fails: zero diagonal during back substitution.")
        x[i] = rhs / Ab[i, i]

    return x

x_gauss = gaussian_solve(A_orig, b_orig)
print("Gaussian solution x =", x_gauss)
print("Residual ||Ax-b||_inf =", residual_inf(A_orig, x_gauss, b_orig))


Gaussian solution x = [0.80338983 0.91525424 1.08474576 1.39661017]
Residual ||Ax-b||_inf = 2.6645352591003757e-15


### Reflection (Gaussian Method)
Gaussian elimination transforms the augmented matrix \([A|b]\) into an upper-triangular form using elementary row operations, then solves by back substitution.
Partial pivoting is important for numerical stability because it avoids division by very small pivot elements.
Limitations: if the matrix is singular or nearly singular, pivots can become close to zero and the method may fail or produce inaccurate results due to floating-point rounding.


# Method 3: Gauss-Jordan

In [7]:
def gauss_jordan_solve(A, b, eps=1e-12):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    n = A.shape[0]

    Ab = np.hstack([A, b.reshape(-1, 1)])

    for col in range(n):
        pivot_row = col + np.argmax(np.abs(Ab[col:, col]))
        if abs(Ab[pivot_row, col]) < eps:
            raise ValueError("Gauss-Jordan fails: pivot is zero (or near zero).")

        if pivot_row != col:
            Ab[[col, pivot_row]] = Ab[[pivot_row, col]]

        pivot = Ab[col, col]
        Ab[col, :] /= pivot  # make pivot = 1

        # eliminate all other rows
        for row in range(n):
            if row == col:
                continue
            factor = Ab[row, col]
            Ab[row, :] -= factor * Ab[col, :]

    return Ab[:, -1]

x_gj = gauss_jordan_solve(A_orig, b_orig)
print("Gauss-Jordan solution x =", x_gj)
print("Residual ||Ax-b||_inf =", residual_inf(A_orig, x_gj, b_orig))


Gauss-Jordan solution x = [0.80338983 0.91525424 1.08474576 1.39661017]
Residual ||Ax-b||_inf = 1.7763568394002505e-15


### Reflection (Gauss–Jordan Method)
Gauss–Jordan elimination reduces \([A|b]\) directly to \([I|x]\), so the solution is obtained without back substitution.
Its limitation is higher computational cost compared to Gaussian elimination because it eliminates entries both below and above the pivot, performing more operations. It is useful for educational purposes and for computing matrix inverses, but Gaussian elimination is often preferred for efficiency.


# Method 4: Jacobi's method

In [10]:
def jacobi_solve(A, b, x0, tol=1e-6, max_iter=100, eps=1e-12):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    x_old = np.array(x0, dtype=float)

    n = A.shape[0]
    if np.any(np.abs(np.diag(A)) < eps):
        raise ValueError("Jacobi fails: zero on diagonal.")

    history = [x_old.copy()]
    errors = []

    for k in range(1, max_iter + 1):
        x_new = np.zeros(n, dtype=float)

        # compute each component using ONLY x_old
        for i in range(n):
            s = np.dot(A[i, :], x_old) - A[i, i] * x_old[i]
            x_new[i] = (b[i] - s) / A[i, i]

        err = rel_error_inf(x_new, x_old)
        history.append(x_new.copy())
        errors.append(err)

        if err < tol:
            return x_new, history, errors, "stopped by tolerance", k

        x_old = x_new

    return x_old, history, errors, "stopped by max_iter", max_iter


x_jac, hist_jac, err_jac, reason_jac, it_jac = jacobi_solve(A, b, x0, tol, max_iter)
df_jac = make_iteration_table(hist_jac, err_jac)

print("Jacobi final x =", x_jac)
print("Stopping reason:", reason_jac, "| iterations:", it_jac, "| tol =", tol, "| max_iter =", max_iter)
print("Residual ||Ax-b||_inf =", residual_inf(A, x_jac, b))
show_table(df_jac, max_rows=15)


Jacobi final x = [0.8033895  0.91525451 1.08474566 1.39661049]
Stopping reason: stopped by tolerance | iterations: 15 | tol = 1e-06 | max_iter = 100
Residual ||Ax-b||_inf = 3.342955960050631e-06


,k,x1,x2,x3,x4,rel_error
0,0,0.000000,0.000000,0.000000,0.000000,NaN
1,1,0.428571,0.833333,1.125000,1.142857,1.000000
2,2,0.713435,0.901786,1.110119,1.306973,0.217957
3,3,0.772230,0.919430,1.086469,1.376458,0.050481
4,4,0.797982,0.913707,1.085600,1.387357,0.018562
5,5,0.800403,0.916037,1.084458,1.395408,0.005770
6,6,0.803199,0.914909,1.084871,1.395604,0.002004
7,7,0.803035,0.915411,1.084687,1.396623,0.000730
8,8,0.803424,0.915183,1.084772,1.396478,0.000279
9,9,0.803338,0.915286,1.084734,1.396634,0.000112


... table truncated (shown first 15 of 16 rows)


### Reflection (Jacobi Method)
Jacobi method updates all variables using only values from the previous iteration.
Convergence is typically guaranteed when the system is diagonally dominant (or under other sufficient conditions).
Limitations: Jacobi can be slow because it does not use updated values within the same iteration. If the matrix is not suitable (e.g., not diagonally dominant), the method may converge very slowly or diverge.


# Method 5: Gauss-Seidel

In [7]:
def gauss_seidel_solve(A, b, x0, tol=1e-6, max_iter=100, eps=1e-12):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    x = np.array(x0, dtype=float)

    n = A.shape[0]
    if np.any(np.abs(np.diag(A)) < eps):
        raise ValueError("Gauss-Seidel fails: zero on diagonal.")

    history = [x.copy()]
    errors = []

    for k in range(1, max_iter + 1):
        x_old = x.copy()

        # update sequentially using the newest values
        for i in range(n):
            s1 = np.dot(A[i, :i], x[:i])          # new values
            s2 = np.dot(A[i, i+1:], x_old[i+1:])  # old values
            x[i] = (b[i] - s1 - s2) / A[i, i]

        err = rel_error_inf(x, x_old)
        history.append(x.copy())
        errors.append(err)

        if err < tol:
            return x, history, errors, "stopped by tolerance", k

    return x, history, errors, "stopped by max_iter", max_iter


x_gs, hist_gs, err_gs, reason_gs, it_gs = gauss_seidel_solve(A, b, x0, tol, max_iter)
df_gs = make_iteration_table(hist_gs, err_gs)

print("Gauss-Seidel final x =", x_gs)
print("Stopping reason:", reason_gs, "| iterations:", it_gs, "| tol =", tol, "| max_iter =", max_iter)
print("Residual ||Ax-b||_inf =", residual_inf(A, x_gs, b))
show_table(df_gs, max_rows=15)


Gauss-Seidel final x = [0.80338984 0.91525423 1.08474576 1.39661017]
Stopping reason: stopped by tolerance | iterations: 8 | tol = 1e-06 | max_iter = 100
Residual ||Ax-b||_inf = 7.771624366270657e-08


,k,x1,x2,x3,x4,rel_error
0,0,0.000000,0.000000,0.000000,0.000000,NaN
1,1,0.428571,0.904762,0.958333,1.272959,1.000000e+00
2,2,0.784621,0.911666,1.072084,1.389951,2.561597e-01
3,3,0.802784,0.914153,1.084127,1.396506,1.300570e-02
4,4,0.803291,0.915152,1.084758,1.396598,7.155444e-04
5,5,0.803370,0.915255,1.084747,1.396605,7.370017e-05
6,6,0.803388,0.915255,1.084745,1.396610,1.295731e-05
7,7,0.803390,0.915254,1.084746,1.396610,1.174291e-06
8,8,0.803390,0.915254,1.084746,1.396610,6.084059e-08


### Reflection (Gauss–Seidel Method)
Gauss–Seidel updates variables sequentially and immediately uses new values in the same iteration:
\[
\begin{cases}
x_i^{(k+1)}=\frac{b_i-\sum_{j<i}a_{ij}x_j^{(k+1)}-\sum_{j>i}a_{ij}x_j^{(k)}}{a_{ii}}
\end{cases}
\]
Because it uses the newest information, it often converges faster than Jacobi.
Limitations: it depends on the order of equations/variables and can still diverge if the matrix does not satisfy convergence conditions. It is also less suitable for parallel computation than Jacobi.


# Method 6: Relaxation method

In [10]:
def sor_solve(A, b, x0, omega=1.9, tol=1e-6, max_iter=100, eps=1e-12):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    x = np.array(x0, dtype=float)

    n = A.shape[0]
    if np.any(np.abs(np.diag(A)) < eps):
        raise ValueError("SOR fails: zero on diagonal.")
    if not (0 < omega < 2):
        raise ValueError("SOR omega must be in (0, 2).")

    history = [x.copy()]
    errors = []

    for k in range(1, max_iter + 1):
        x_old = x.copy()

        for i in range(n):
            s1 = np.dot(A[i, :i], x[:i])            # new
            s2 = np.dot(A[i, i+1:], x_old[i+1:])    # old
            x_gs = (b[i] - s1 - s2) / A[i, i]  # GS step for i
            x[i] = (1 - omega) * x_old[i] + omega * x_gs

        err = rel_error_inf(x, x_old)
        history.append(x.copy())
        errors.append(err)

        if err < tol:
            return x, history, errors, "stopped by tolerance", k

    return x, history, errors, "stopped by max_iter", max_iter


omega = 1.9
x_sor, hist_sor, err_sor, reason_sor, it_sor = sor_solve(A, b, x0, omega, tol, max_iter)
df_sor = make_iteration_table(hist_sor, err_sor)

print("SOR final x =", x_sor, "| omega =", omega)
print("Stopping reason:", reason_sor, "| iterations:", it_sor, "| tol =", tol, "| max_iter =", max_iter)
print("Residual ||Ax-b||_inf =", residual_inf(A, x_sor, b))
show_table(df_sor, max_rows=15)


SOR final x = [ 24.45686253  36.70128824 -32.34148107 -39.83209139] | omega = 1.9
Stopping reason: stopped by max_iter | iterations: 100 | tol = 1e-06 | max_iter = 100
Residual ||Ax-b||_inf = 266.69559545393645


,k,x1,x2,x3,x4,rel_error
0,0,0.000000,0.000000,0.000000,0.000000,NaN
1,1,0.814286,1.841190,1.506824,2.522713,1.000000
2,2,1.541658,0.092756,0.992329,0.982057,1.134127
3,3,-0.284260,1.413091,1.209545,1.078017,1.292145
4,4,1.710577,0.894885,0.686141,2.073153,0.962223
5,5,0.456852,0.483386,1.789040,0.907988,0.700781
6,6,0.541635,1.598803,0.234657,1.278002,0.972217
7,7,1.390855,0.254455,1.839073,2.206373,0.727174
8,8,0.330151,1.342560,0.609079,0.165830,1.519890
9,9,0.806259,0.770707,1.254184,2.591095,0.936000


... table truncated (shown first 15 of 101 rows)


### Reflection (Relaxation / SOR Method)
SOR (Successive Over-Relaxation) modifies Gauss–Seidel by adding a relaxation parameter \(\omega\):
\[
\begin{cases}
x_i^{(k+1)}=(1-\omega)x_i^{(k)}+\omega x_{GS,i}^{(k+1)}
\end{cases}
\]
For \(\omega=1\), SOR becomes Gauss–Seidel. Values \(1<\omega<2\) can accelerate convergence.
Limitations: choosing \(\omega\) is crucial. A poor \(\omega\) can slow convergence or even cause divergence. Therefore, SOR is powerful but requires parameter tuning.


In [9]:
# Collect results
results = []

def add_result(name, x, A_use, b_use, iters, reason):
    results.append({
        "Method": name,
        "Iterations": iters,
        "Stopping": reason,
        "Residual_inf": residual_inf(A_use, x, b_use),
        "x1": x[0], "x2": x[1], "x3": x[2], "x4": x[3]
    })

add_result("Cramer", x_cramer, A_orig, b_orig, 0, "direct")
add_result("Gaussian", x_gauss, A_orig, b_orig, 0, "direct")
add_result("Gauss-Jordan", x_gj, A_orig, b_orig, 0, "direct")
add_result("Jacobi", x_jac, A, b, it_jac, reason_jac)
add_result("Gauss-Seidel", x_gs, A, b, it_gs, reason_gs)
add_result(f"SOR (ω={omega})", x_sor, A, b, it_sor, reason_sor)

df_comp = pd.DataFrame(results)
display(df_comp)


,Method,Iterations,Stopping,Residual_inf,x1,x2,x3,x4
0,Cramer,0,direct,7.105427e-15,0.803390,0.915254,1.084746,1.39661
1,Gaussian,0,direct,2.664535e-15,0.803390,0.915254,1.084746,1.39661
2,Gauss-Jordan,0,direct,1.776357e-15,0.803390,0.915254,1.084746,1.39661
3,Jacobi,15,stopped by tolerance,3.342956e-06,0.803389,0.915255,1.084746,1.39661
4,Gauss-Seidel,8,stopped by tolerance,7.771624e-08,0.803390,0.915254,1.084746,1.39661
5,SOR (ω=1.1),9,stopped by tolerance,1.119614e-06,0.803390,0.915254,1.084746,1.39661


## Comparison Summary
Direct methods (Cramer, Gaussian, Gauss–Jordan) produced the solution in a finite number of operations without iterations. Among direct methods, Gaussian elimination is generally the most practical due to good efficiency and numerical stability with partial pivoting.

For iterative methods, the system was rearranged to achieve diagonal dominance, which supports convergence. Using the same initial guess \(x^{(0)}=(0,0,0,0)\), tolerance \(10^{-6}\), and maximum iterations 100:
- **Jacobi** converged but required more iterations because it uses only previous-iteration values.
- **Gauss–Seidel** converged faster than Jacobi since it uses updated values immediately.
- **SOR (ω = 1.1)** converged the fastest in this experiment, because relaxation can accelerate convergence when ω is chosen appropriately.

The residual norms \(\|Ax-b\|_\infty\) confirm that all methods achieved accurate solutions (small residuals).
Potential difficulties include divergence or slow convergence if diagonal dominance is not satisfied and sensitivity of SOR to the choice of ω.


## Overall Conclusion
This notebook implemented three direct methods (Cramer, Gaussian, Gauss–Jordan) and three iterative methods (Jacobi, Gauss–Seidel, SOR) for solving a 4×4 system of linear algebraic equations.

Direct methods provide a deterministic solution in a fixed number of steps, and Gaussian elimination with partial pivoting is the most suitable in practice for efficiency and stability. Cramer’s method is correct but computationally expensive and not practical for large systems.

Iterative methods are useful for large sparse systems, but they require convergence conditions. After rearranging the equations to ensure diagonal dominance, Jacobi, Gauss–Seidel, and SOR converged using the same initial guess and stopping criteria. Gauss–Seidel was faster than Jacobi, and SOR achieved the best convergence speed for ω = 1.1.

In summary, for small dense systems, Gaussian elimination is recommended, while for large systems iterative methods can be preferable if convergence conditions are satisfied.

